In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LinearRegression, ElasticNet, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, Stepwise, sklearn_selected, sklearn_selection_path)
from functools import partial
from l0bnb import fit_path

In [2]:
Hitters = load_data('Hitters')
np.isnan(Hitters['Salary']).sum()

59

In [3]:
Hitters = Hitters.dropna(how='any')
Hitters.shape

(263, 20)

In [4]:
def nCp(sigma2, estimator, X, y):
    n, p = X.shape
    y_hat = estimator.predict(X)
    RSS = np.sum((y - y_hat) ** 2)
    return -(RSS + 2*p*sigma2) / n

In [5]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
y = np.array(Hitters['Salary'])
X = design.transform(Hitters)
sigma2 = sm.OLS(y, X).fit().scale

In [6]:
neg_Cp = partial(nCp, sigma2)

In [7]:
strategy = Stepwise.first_peak(design, direction='forward', max_terms=len(design.terms))

In [8]:
hitters_Cp = sklearn_selected(sm.OLS, strategy, scoring=neg_Cp)
hitters_Cp.fit(Hitters, y)
hitters_Cp.selected_state_

('Assists',
 'AtBat',
 'CAtBat',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Hits',
 'PutOuts',
 'Walks')

In [ ]:
ridge = ElasticNet(l1_ratio=0, alpha=0.1)
scaler = StandardScaler()
pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
pipe.fit(X, y)

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

Pipeline(steps=[('scaler', StandardScaler()),
                ('ridge', ElasticNet(alpha=0.1, l1_ratio=0))])

In [13]:
pipe?

Type:        Pipeline
String form:
Pipeline(steps=[('scaler', StandardScaler()),
                ('ridge', ElasticNet(alpha=0.1, l1_ratio=0))])
Length:      2
File:        c:\users\admin\anaconda3\lib\site-packages\sklearn\pipeline.py
Docstring:  
Pipeline of transforms with a final estimator.

Sequentially apply a list of transforms and a final estimator.
Intermediate steps of the pipeline must be 'transforms', that is, they
must implement `fit` and `transform` methods.
The final estimator only needs to implement `fit`.
The transformers in the pipeline can be cached using ``memory`` argument.

The purpose of the pipeline is to assemble several steps that can be
cross-validated together while setting different parameters. For this, it
enables setting parameters of the various steps using their names and the
parameter name separated by a `'__'`, as in the example below. A step's
estimator may be replaced entirely by setting the parameter with its name
to another estimator, or a transfor

In [18]:
param_grid = {'ridge__alpha':[0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X, y)
grid.best_params_, grid.best_estimator_, grid.best_score_

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

({'ridge__alpha': 0.01},
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('ridge', ElasticNet(alpha=0.01, l1_ratio=0))]),
 -119071.14154557462)

In [24]:
lassoCV = ElasticNetCV(n_alphas=100, l1_ratio=1, cv=10)
pipeCV = Pipeline(steps=[('scaler', scaler), ('lasso', lassoCV)])
pipeCV.fit(X, y)
tuned_lasso = pipeCV.named_steps['lasso']
tuned_lasso.alpha_, tuned_lasso.coef_, tuned_lasso.get_params()

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

(2.552820965069261,
 array([   0.        , -234.40024002,  259.94594324,    0.        ,
          -0.        ,    0.        ,  104.14775303,  -45.97504202,
          -0.        ,    0.        ,   45.04856657,  223.55406425,
         122.90647787, -144.16002032,   16.19350779,  -59.56477236,
          76.48997742,   26.1470852 ,  -13.79625172,   -0.        ]),
 {'alphas': None,
  'copy_X': True,
  'cv': 10,
  'eps': 0.001,
  'fit_intercept': True,
  'l1_ratio': 1,
  'max_iter': 1000,
  'n_alphas': 100,
  'n_jobs': None,
  'positive': False,
  'precompute': 'auto',
  'random_state': None,
  'selection': 'cyclic',
  'tol': 0.0001,
  'verbose': 0})

In [26]:
pca = PCA(n_components=2)
linreg = LinearRegression()
scaler = StandardScaler()
pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('linreg', linreg)])
pipe.fit(X, y)
pipe.named_steps['linreg'].coef_

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

array([106.36859204, -21.60350456])

In [27]:
param_grid = {'pca__n_components':range(1, 10)}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X, y)
grid.cv_results_

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

{'mean_fit_time': array([0.00780053, 0.00680637, 0.00620551, 0.00660591, 0.00880775,
        0.00740652, 0.00660644, 0.00640602, 0.00700645]),
 'std_fit_time': array([0.00097237, 0.00074903, 0.00040081, 0.00048992, 0.00160168,
        0.0008007 , 0.00048967, 0.00049021, 0.00063316]),
 'mean_score_time': array([0.00280228, 0.00240216, 0.00280275, 0.00240192, 0.00300326,
        0.00260401, 0.00280275, 0.00260205, 0.00260243]),
 'std_score_time': array([0.00040002, 0.00049049, 0.00040061, 0.0004901 , 0.00063301,
        0.00048882, 0.0004005 , 0.00049052, 0.00049064]),
 'param_pca__n_components': masked_array(data=[1, 2, 3, 4, 5, 6, 7, 8, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'params': [{'pca__n_components': 1},
  {'pca__n_components': 2},
  {'pca__n_components': 3},
  {'pca__n_components': 4},
  {'pca__n_components': 5},
  {'pca__n_components': 6},
  {'pca__n_compon

In [28]:
pipe.named_steps['pca'].explained_variance_ratio_

array([0.3831424 , 0.21841076])

In [29]:
pls = PLSRegression(n_components=2, scale=True)
pls.fit(X, y)

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


PLSRegression()